In [22]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [23]:
class BatsmanState(TypedDict):
    name: str
    runs: int
    balls_faced: int
    fours: int
    sixes: int

    sr: float  # strike rate
    bpb: float  # balls per boundary
    boundary_percent: float  # percentage of runs from boundaries
    summary: str

## How LangGraph Updates State

**Important Concept**: When you return a dictionary from a node function, LangGraph **automatically merges** it into the state.

### What happens under the hood:

```python
# When you write:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    sr = (state['runs'] / state['balls_faced']) * 100
    return {'sr': sr}  # ← Return partial dict

# LangGraph does this internally:
# 1. Calls your function: result = calculate_sr(current_state)
# 2. Merges the result: current_state.update(result)
# 3. Passes updated state to next node
```

### Execution Flow:

1. **Input State**: `{'name': 'Virat', 'runs': 82, 'balls_faced': 53, 'fours': 8, 'sixes': 2}`

2. **After calculate_sr** (parallel): State gets `sr` field added
   - `{'name': 'Virat', 'runs': 82, ..., 'sr': 154.72}`

3. **After calculate_bpb** (parallel): State gets `bpb` field added
   - `{'name': 'Virat', 'runs': 82, ..., 'sr': 154.72, 'bpb': 5.30}`

4. **After calculate_boundary_percent** (parallel): State gets `boundary_percent` field added
   - `{'name': 'Virat', ..., 'sr': 154.72, 'bpb': 5.30, 'boundary_percent': 53.66}`

5. **After summary**: State gets `summary` field added
   - Final complete state with all fields populated

**The merge happens automatically in the `workflow.compile()` execution engine!**

In [24]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    sr = (state['runs'] / state['balls_faced']) * 100 if state['balls_faced'] > 0 else 0
    return {'sr': sr}

In [25]:
def calculate_bpb(state: BatsmanState) -> BatsmanState:
    total_boundaries = state['fours'] + state['sixes']
    bpb= (state['balls_faced'] / total_boundaries) if total_boundaries > 0 else float('inf')
    return {'bpb': bpb}

In [26]:
def calculate_boundary_percent(state: BatsmanState) -> BatsmanState:
    boundary_runs = (state['fours'] * 4) + (state['sixes'] * 6)
    boundary_percent = (boundary_runs / state['runs']) * 100 if state['runs'] > 0 else 0
    return {'boundary_percent': boundary_percent}

In [27]:
def summary(state: BatsmanState) -> BatsmanState:
    summary_text = f"{state['name']} scored {state['runs']} runs off {state['balls_faced']} balls "
    summary_text += f"with a strike rate of {state['sr']:.2f}, "
    summary_text += f"balls per boundary of {state['bpb']:.2f}, "
    summary_text += f"and {state['boundary_percent']:.2f}% of runs from boundaries."
    return {'summary': summary_text}

In [28]:
graph = StateGraph(BatsmanState)

graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percent', calculate_boundary_percent)
graph.add_node('summary', summary)

graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percent', 'summary')

graph.add_edge('summary', END)

workflow = graph.compile()

In [29]:
# Display the graph structure as ASCII text
print(workflow.get_graph().draw_ascii())

                                        +-----------+                                 
                                       *| __start__ |*                                
                                  ***** +-----------+ *****                           
                            ******              *          ******                     
                       *****                     *               *****                
                    ***                          *                    ***             
+----------------------------+           +---------------+           +--------------+ 
| calculate_boundary_percent |           | calculate_bpb |           | calculate_sr | 
+----------------------------+           +---------------+       ****+--------------+ 
                            ******              *          ******                     
                                  *****        *      *****                           
                                       *** 

In [30]:
# Test the workflow with sample data
input_state = {
    'name': 'Virat Kohli',
    'runs': 82,
    'balls_faced': 53,
    'fours': 8,
    'sixes': 2
}

result = workflow.invoke(input_state)
print(result['summary'])

Virat Kohli scored 82 runs off 53 balls with a strike rate of 154.72, balls per boundary of 5.30, and 53.66% of runs from boundaries.
